<a href="https://colab.research.google.com/github/SamuelBFG/DL-studies/blob/master/4_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras Tuner

With so many potential combinations of hyperparameters to tune, coming up with the best model can be a tedious process. Often two or more parameters might have correlated effects on the overall speed of convergence as well as validation accuracy, so tuning one at a time might not lead to the best model.  And if curiosity gets the best of us, we might want to experimentation on all the hyperparameters together!  

Keras Tuner comes to automate this hyperparameter search. We define a search algorithm, the potential values that each parameter can take (e.g. discrete values or a range), our target object to maximize (e.g. validation accuracy) and sit back to see the program start training. Keras Tuner will conduct multiple experiments changing the parameters on our behalf, storing the metadata of the best model. The following code example adapted from Keras Tuner documentation showcases searching through the different model architectures (varying in the number of layers between 2 and 10) as well as varying the learning rate (between 0.1 and 0.001).

In [1]:
!pip install keras-tuner

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=763e89460c27ea5d68f5d4cd09a129ff135ee1af3e5b1bbaad090314859dde32
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=26817c985eddb972c91cfd50b038f9edf861e0bb2e4f4fa1fe184404b10b35bc
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters
from kerastuner.tuners import RandomSearch

In [3]:
# Input data
(x, y), (val_x, val_y) = keras.datasets.mnist.load_data()
x = x.astype('float32') / 255.
val_x = val_x.astype('float32') / 255.

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
# Defining hyper parameters
hp = HyperParameters()
hp.Choice('learning_rate', [0.1, 0.001])
hp.Int('num_layers', 2, 10)

2

In [5]:
#Defining model with expandable number of layers
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    for i in range(hp.get('num_layers')):
        model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(hp.get('learning_rate')),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [6]:
hypermodel = RandomSearch(
    build_model,
    max_trials=20,  #Number of combinations allowed
    hyperparameters=hp,
    allow_new_entries=False,
    objective='val_accuracy')

In [7]:
hypermodel.search(x=x, y=y, epochs=5, validation_data=(val_x, val_y))

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4110 - accuracy: 0.8714 - val_loss: 0.2013 - val_accuracy: 0.9388
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1827 - accuracy: 0.9471 - val_loss: 0.1825 - val_accuracy: 0.9464
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1461 - accuracy: 0.9572 - val_loss: 0.1593 - val_accuracy: 0.9522
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1268 - accuracy: 0.9633 - val_loss: 0.1395 - val_accuracy: 0.9582
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1122 - accuracy: 0.9676 - val_loss: 0.1421 - val_accuracy: 0.9594


Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3707 - accuracy: 0.8864 - val_loss: 0.1766 - val_accuracy: 0.9483
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1735 - accuracy: 0.9482 - val_loss: 0.1513 - val_accuracy: 0.9548
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1376 - accuracy: 0.9591 - val_loss: 0.1305 - val_accuracy: 0.9629
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1196 - accuracy: 0.9640 - val_loss: 0.1810 - val_accuracy: 0.9471
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1055 - accuracy: 0.9683 - val_loss: 0.1222 - val_accuracy: 0.9645


Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3421 - accuracy: 0.8955 - val_loss: 0.1768 - val_accuracy: 0.9464
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1596 - accuracy: 0.9526 - val_loss: 0.1441 - val_accuracy: 0.9564
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1267 - accuracy: 0.9614 - val_loss: 0.1505 - val_accuracy: 0.9530
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1074 - accuracy: 0.9672 - val_loss: 0.1270 - val_accuracy: 0.9610
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0932 - accuracy: 0.9717 - val_loss: 0.1070 - val_accuracy: 0.9673


Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.3387 - accuracy: 0.1029 - val_loss: 2.3164 - val_accuracy: 0.1135
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.3145 - accuracy: 0.1023 - val_loss: 2.3105 - val_accuracy: 0.0974
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 2.3146 - accuracy: 0.1037 - val_loss: 2.3069 - val_accuracy: 0.1135
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3140 - accuracy: 0.1042 - val_loss: 2.3111 - val_accuracy: 0.1135
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.3147 - accuracy: 0.1033 - val_loss: 2.3191 - val_accuracy: 0.1135


Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3725 - accuracy: 0.1028 - val_loss: 2.3086 - val_accuracy: 0.0958
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3144 - accuracy: 0.1035 - val_loss: 2.3148 - val_accuracy: 0.1135
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3138 - accuracy: 0.1031 - val_loss: 2.3137 - val_accuracy: 0.0958
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3133 - accuracy: 0.1050 - val_loss: 2.3151 - val_accuracy: 0.0958
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3144 - accuracy: 0.1039 - val_loss: 2.3084 - val_accuracy: 0.0982


...

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 1.6607 - accuracy: 0.3395 - val_loss: 1.5291 - val_accuracy: 0.3782
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 1.5854 - accuracy: 0.3541 - val_loss: 1.5109 - val_accuracy: 0.3533
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 1.7959 - accuracy: 0.2862 - val_loss: 2.3518 - val_accuracy: 0.1010
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 2.3186 - accuracy: 0.1032 - val_loss: 2.3164 - val_accuracy: 0.1135
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 2.3138 - accuracy: 0.1019 - val_loss: 2.3182 - val_accuracy: 0.0982


Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3581 - accuracy: 0.8902 - val_loss: 0.1849 - val_accuracy: 0.9435
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1718 - accuracy: 0.9488 - val_loss: 0.1561 - val_accuracy: 0.9517
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1378 - accuracy: 0.9589 - val_loss: 0.1460 - val_accuracy: 0.9557
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1198 - accuracy: 0.9639 - val_loss: 0.1356 - val_accuracy: 0.9574
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1051 - accuracy: 0.9676 - val_loss: 0.1300 - val_accuracy: 0.9612


Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.3064 - accuracy: 0.1253 - val_loss: 2.3166 - val_accuracy: 0.1028
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.3135 - accuracy: 0.1014 - val_loss: 2.3076 - val_accuracy: 0.1032
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.3146 - accuracy: 0.1029 - val_loss: 2.3174 - val_accuracy: 0.0982
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.3136 - accuracy: 0.1013 - val_loss: 2.3121 - val_accuracy: 0.1010
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 2.3147 - accuracy: 0.1031 - val_loss: 2.3143 - val_accuracy: 0.1010


Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4868 - accuracy: 0.8481 - val_loss: 0.2433 - val_accuracy: 0.9279
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2070 - accuracy: 0.9393 - val_loss: 0.1950 - val_accuracy: 0.9418
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1622 - accuracy: 0.9528 - val_loss: 0.1772 - val_accuracy: 0.9492
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1393 - accuracy: 0.9592 - val_loss: 0.1650 - val_accuracy: 0.9541
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1249 - accuracy: 0.9628 - val_loss: 0.1481 - val_accuracy: 0.9594


INFO:tensorflow:Oracle triggered exit


In [8]:
# Show summary of overall best model
hypermodel.results_summary()